# How are organisations identified in 360Giving's GrantNav? 

**Author**: Edafe Onerhime

**Last Updated**: 2017-07-11

**Description**:  Analyses the organisation id's used in [GrantNav](http://grantnav.threesixtygiving.org) using the prefixes in the **Recipient Org: Identifier** field.

**Contents**: This Jupyter Notebook, data folder

## About

This is an experiment to understand how funding organisations talk about the recipients of their grants by analysing the identifiers they provide for them. 


**Why identifiers matter**

Identifiers are an important part of any dataset. They let a computer uniquely identify and refer to specific grants, organisations, transactions and so-on. 

Whilst a human being may be good at recognising that:

>INDIGO TRUST, The Indigo Trust, and indigo-trust

... all refer to the same organisation, computers find this a lot trickier.

Identifiers can be valuable for joining up pools of information and improving the quality of the information we hold.

**What will this analysis tell us?**

We can better understand what identifiers publishers prefer, if we should provide some interventions and where to target our efforts.

For example, do publishers prefer their own internal reference? Can we understand why? Do publishers prefer charity numbers over company numbers? How can we support them while making the information more widely useful?

## Instructions


## Things to note


## Things to do



In [1]:
import os.path
import pandas as pd

In [2]:
fileDetails = {'grantnav.csv': 'http://grantnav.threesixtygiving.org/search.csv', 
               'org-id.csv': 'http://org-id.guide/download.csv'}

In [3]:
# Get GrantNav and Org-Id files
for fname in fileDetails:
    print('Checking for:',fname)
    if not os.path.isfile(fname):
        print('Downloading:',fname)
        urlretrieve(fileDetails[fname], fname)
        

Checking for: org-id.csv
Checking for: grantnav.csv


In [4]:
# =======================
# Process GrantNav data
# =======================

# Load files
dfGN = pd.read_csv('grantnav.csv',usecols=['Recipient Org:Identifier'])

# Split column 
# (Replace anything containing 360G with 360G- to group all publisher-own prefixes)
dfGN.loc[dfGN['Recipient Org:Identifier'].str.contains('360G'), 'Recipient Org:Identifier'] = '360G'
dfGN = pd.DataFrame(dfGN['Recipient Org:Identifier'].str.split('-',2).tolist(),columns = ['split1','split2','discard']).fillna('')
dfGN['code'] = dfGN['split1']+'-'+dfGN['split2']
dfGN = dfGN[['code']]
dfGN['code'] = dfGN.code.str.strip('-')
dfGN = pd.DataFrame({'count' : dfGN.groupby( ['code'] ).size()}).reset_index()

In [20]:
# =======================
# Process GrantNav data
# =======================

# Load files
dfOID = pd.read_csv('org-id.csv',usecols=['code', 'quality', "name/en"])
dfOID.rename(columns={"name/en": 'name'}, inplace=True)

In [24]:
# ========================
# Merge & export datasets
# ========================

df = pd.merge(dfGN, dfOID, how='left', on=['code'])
df.loc[df['code'].str.contains('360G'), 'name'] = "Publisher's internal reference"
df[['code',"name", 'quality','count']].fillna('').to_csv('grantnav_recipient_prefix.csv',index=False)